记录鼠标点击位置

In [ ]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
#author:June_xu
import pyautogui as mk
import time,pythoncom,os,shelve,subprocess,sys
import PyHook3 as pyHook

# print(os.getcwd())
click_position_records={}
def read_records_log():
    global f,click_position_records
    f=shelve.open("mouseRecords")
    if f:
        click_position_records=f.get("click_position_records")

click_timer=1
def record_mouse(event):
    global click_position_records, click_timer,f,q
    if event.Message==513:
        x,y=event.Position
        color=mk.screenshot().getpixel((x,y))
        click_info=[x,y,color[0],color[1],color[2]]
        click_position_records[click_timer] = click_info
        print("第%s次点击 Position：(%s,%s) color:%s"%(click_timer,x,y,color))
        click_timer += 1
        return True
    elif event.Message==519:
        f["click_position_records"]=click_position_records
        print("退出记录器")
        myName=os.path.basename(sys.argv[0]).split(".")[0]
        time.sleep(1)
        subprocess.call("taskkill /f /im %s /t"%"python.exe")
        raise Exception("记录终止")


def listen_mouse_event():
    global click_position_records,click_timer,listener_switch
    listener = pyHook.HookManager()
    listener.MouseAllButtonsDown=record_mouse
    listener.HookMouse()
    pythoncom.PumpMessages()


read_records_log()
listen_mouse_event()